In [1]:
import pandas as pd
import numpy as np
import requests
import yfinance
import re

from newspaper import Article
from datetime import datetime
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [2]:
finance = pd.read_csv('./datasets/finance.csv')
finance = finance.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
finance

,sentiment,headline_en,headline_ru
0,neutral,"According to Gran , the company has no plans t...","По словам Граня, компания не планирует перенос..."
1,neutral,Technopolis plans to develop in stages an area...,«Технополис» планирует поэтапно застроить терр...
2,negative,The international electronic industry company ...,Международная компания электронной промышленно...
3,positive,With the new production plant the company woul...,Благодаря новому производственному предприятию...
4,positive,According to the company 's updated strategy f...,Согласно обновленной стратегии компании на 200...
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,LONDON MarketWatch – Цены на акции в Лондоне в...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"Продажи пива Rinkuskiai упали на 6,5% до 4,16 ..."
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"Операционная прибыль упала до 35,4 млн евро с ..."
4844,negative,Net sales of the Paper segment decreased to EU...,Чистые продажи бумажного сегмента снизились до...


In [3]:
gpt_train = pd.read_csv('./datasets/gpt_train.csv')
gpt_train = gpt_train.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
gpt_train

,headline_en,sentiment,headline_ru
0,Investors rejoice at the record growth of bitcoin,positive,Инвесторы радуются рекордному росту биткоина
1,Experts warn about the risks of investing in c...,negative,Эксперты предупреждают о рисках инвестирования...
2,"Bitcoin is updating its maximum, attracting ne...",positive,"Биткойн обновляет свой максимум, привлекая нов..."
3,Bitcoin transactions are subject to cyber attacks,negative,Биткойн-транзакции подвергаются кибератакам
4,Developers have introduced new technologies to...,positive,Разработчики представили новые технологии для ...
...,...,...,...
999,Investors assess the role of cryptocurrencies ...,positive,Инвесторы оценивают роль криптовалют в содейст...
1000,Cryptocurrencies are gaining popularity in the...,positive,Криптовалюты набирают популярность в сфере отв...
1001,Bitcoin is becoming part of a long-term invest...,positive,Биткойн становится частью долгосрочной инвести...
1002,Experts are considering the impact of cryptocu...,neutral,Эксперты рассматривают влияние криптовалют на ...


In [4]:
risks = pd.read_csv('./datasets/risks.csv')
risks = risks.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
risks

,headline_en,sentiment,headline_ru
0,The GeoSolutions technology will leverage Bene...,positive,Технология GeoSolutions будет использовать GPS...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,"$ESI находится на минимуме, падение с $1,50 до..."
2,"For the last quarter of 2010 , Componenta 's n...",positive,За последний квартал 2010 года чистый объем пр...
3,According to the Finnish-Russian Chamber of Co...,neutral,"По данным Финско-Российской торговой палаты, в..."
4,The Swedish buyout firm has sold its remaining...,neutral,Шведская фирма по выкупу продала оставшиеся 22...
...,...,...,...
5837,RISING costs have forced packaging producer Hu...,negative,РОСТ затрат вынудил производителя упаковки Huh...
5838,Nordic Walking was first used as a summer trai...,neutral,Скандинавская ходьба впервые использовалась в ...
5839,"According shipping company Viking Line , the E...",neutral,"По мнению судоходной компании Viking Line, реш..."
5840,"In the building and home improvement trade , s...",neutral,В сфере строительства и товаров для дома прода...


In [5]:
user_responses = pd.read_csv('./datasets/user_respsonses.csv')
user_responses = user_responses.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
user_responses

,headline_en,sentiment,headline_ru
0,It isn't an investment - it's speculation in a...,negative,Это не инвестиции – это спекуляция в финансово...
1,Joke of the decade,negative,Шутка десятилетия
2,This is pretty big news. Love to see utility e...,positive,Это довольно большая новость. Приятно видеть п...
3,"Just hold your bag and have an exit strategy, ...",positive,Просто держите свою сумку и имейте стратегию в...
4,While you can trace a common thread from the h...,negative,Хотя вы можете проследить общую нить от контрк...
...,...,...,...
557,Glorified ponzi scheme that wastes people's ti...,negative,"Прославленная финансовая пирамида, которая тра..."
558,Axie will fail because it’s just a mediocre ga...,negative,"Axie потерпит неудачу, потому что это всего ли..."
559,I do not feel sorry for anyone who got into Po...,negative,"Мне не жаль никого, кто попал в ПонциКвон II"
560,good time to accumulate,positive,хорошее время для накопления


In [6]:
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def lower(text):
    low_text= text.lower()
    return low_text

def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_mention(x):
    text=re.sub(r'@\w+','',x)
    return text

def remove_hash(x):
    text=re.sub(r'#\w+','',x)
    return text

def remove_space(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove

In [7]:
", ".join(stopwords.words('russian'))
STOPWORDS = set(stopwords.words('russian'))

def remove_urls_ru(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html_ru(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def lower_ru(text):
    low_text= text.lower()
    return low_text

def remove_num_ru(text):
    remove= re.sub(r'\d+', '', text)
    return remove

def punct_remove_ru(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct

def remove_stopwords_ru(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_mention_ru(x):
    text=re.sub(r'@\w+','',x)
    return text

def remove_hash_ru(x):
    text=re.sub(r'#\w+','',x)
    return text

def remove_space_ru(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove

In [8]:
df = pd.concat([finance, gpt_train, user_responses, risks], ignore_index=True)
df

,sentiment,headline_en,headline_ru
0,neutral,"According to Gran , the company has no plans t...","По словам Граня, компания не планирует перенос..."
1,neutral,Technopolis plans to develop in stages an area...,«Технополис» планирует поэтапно застроить терр...
2,negative,The international electronic industry company ...,Международная компания электронной промышленно...
3,positive,With the new production plant the company woul...,Благодаря новому производственному предприятию...
4,positive,According to the company 's updated strategy f...,Согласно обновленной стратегии компании на 200...
...,...,...,...
12249,negative,RISING costs have forced packaging producer Hu...,РОСТ затрат вынудил производителя упаковки Huh...
12250,neutral,Nordic Walking was first used as a summer trai...,Скандинавская ходьба впервые использовалась в ...
12251,neutral,"According shipping company Viking Line , the E...","По мнению судоходной компании Viking Line, реш..."
12252,neutral,"In the building and home improvement trade , s...",В сфере строительства и товаров для дома прода...


In [9]:
def nlp_process_ru(df):
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_urls_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_html_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:lower_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_num_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:punct_remove_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_stopwords_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_mention_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_hash_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_space_ru(x))
    return df

def nlp_process(df):
    df['headline_en']=df['headline_en'].apply(lambda x:remove_urls(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_html(x))
    df['headline_en']=df['headline_en'].apply(lambda x:lower(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_num(x))
    df['headline_en']=df['headline_en'].apply(lambda x:punct_remove(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_stopwords(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_mention(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_hash(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_space(x))
    return df

In [10]:
df = nlp_process(df)
df = nlp_process_ru(df)
df

,sentiment,headline_en,headline_ru
0,neutral,according to gran the company has no plans to ...,словам граня компания планирует переносить про...
1,neutral,technopolis plans to develop in stages an area...,технополис планирует поэтапно застроить террит...
2,negative,the international electronic industry company ...,международная компания электронной промышленно...
3,positive,with the new production plant the company woul...,благодаря новому производственному предприятию...
4,positive,according to the company s updated strategy fo...,согласно обновленной стратегии компании годы b...
...,...,...,...
12249,negative,rising costs have forced packaging producer hu...,рост затрат вынудил производителя упаковки huh...
12250,neutral,nordic walking was first used as a summer trai...,скандинавская ходьба впервые использовалась ка...
12251,neutral,according shipping company viking line the eu ...,мнению судоходной компании viking line решение...
12252,neutral,in the building and home improvement trade sal...,сфере строительства товаров дома продажи снизи...


In [11]:
ru_df = df[['headline_ru', 'sentiment']]
en_df = df[['headline_en', 'sentiment']]
df = pd.DataFrame({"headline": pd.concat([en_df['headline_en'], ru_df['headline_ru']], ignore_index=True), "sentiment": pd.concat([en_df['sentiment'], ru_df['sentiment']], ignore_index=True)})
df

,headline,sentiment
0,according to gran the company has no plans to ...,neutral
1,technopolis plans to develop in stages an area...,neutral
2,the international electronic industry company ...,negative
3,with the new production plant the company woul...,positive
4,according to the company s updated strategy fo...,positive
...,...,...
24503,рост затрат вынудил производителя упаковки huh...,negative
24504,скандинавская ходьба впервые использовалась ка...,neutral
24505,мнению судоходной компании viking line решение...,neutral
24506,сфере строительства товаров дома продажи снизи...,neutral


In [12]:
df = df.dropna()
null_count = df.isnull().sum()
null_count

headline     0
sentiment    0
dtype: int64

In [13]:
df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df

,headline,sentiment
0,on the other hand montevideo said that having ...,positive
1,августеоктябре года результат компании уплаты ...,positive
2,msft sql server revenue grew doubledigit with ...,positive
3,currently glaston employs approximately persons,neutral
4,companiesmeggitt profits hit by weak energy an...,negative
...,...,...
24503,finnish fibrebased materials maker ahlstrom co...,neutral
24504,operating profit decreased to eur mn from eur ...,neutral
24505,investors are looking for alternative assets i...,positive
24506,in the group s net sales stood at eur million ...,neutral


In [14]:
X = df['headline']
y = df['sentiment']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=X_padded.shape[1]))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax')) 

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")
model.save('sentiment_model_v5.keras')

Epoch 1/50
246/246 [==============================] - 1s 5ms/step - loss: 1.0007 - accuracy: 0.5145 - val_loss: 0.9749 - val_accuracy: 0.5393
Epoch 2/50
246/246 [==============================] - 1s 5ms/step - loss: 0.9920 - accuracy: 0.5145 - val_loss: 0.9753 - val_accuracy: 0.5393
Epoch 3/50
246/246 [==============================] - 1s 6ms/step - loss: 0.9836 - accuracy: 0.5156 - val_loss: 0.9673 - val_accuracy: 0.5586
Epoch 4/50
246/246 [==============================] - 2s 6ms/step - loss: 0.9240 - accuracy: 0.5545 - val_loss: 0.8711 - val_accuracy: 0.5908
Epoch 5/50
246/246 [==============================] - 1s 6ms/step - loss: 0.8227 - accuracy: 0.6343 - val_loss: 0.8101 - val_accuracy: 0.6242
Epoch 6/50
246/246 [==============================] - 2s 6ms/step - loss: 0.7430 - accuracy: 0.6817 - val_loss: 0.7747 - val_accuracy: 0.6843
Epoch 7/50
246/246 [==============================] - 1s 6ms/step - loss: 0.6801 - accuracy: 0.7178 - val_loss: 0.7354 - val_accuracy: 0.6925
Epoch 

In [15]:
check_df = pd.read_csv('./datasets/check.csv')
check_df = check_df.drop(columns=['id'])
check_df = nlp_process(check_df)
check_df = nlp_process_ru(check_df)
ru_check_df = check_df['headline_ru']
en_check_df = check_df['headline_en']
check_df = pd.DataFrame({"headline": pd.concat([en_check_df, ru_check_df], ignore_index=True)})
check_df

,headline
0,биткоин упал последние часа вызвав тревогу сре...
1,биткоин достиг нового исторического максимума ...
2,биткоин останется таким году
3,президент франции заявляет концу года купит би...
4,последние новостные сообщения отмечают увеличе...
5,последние колебания цене биткоина вызвали опас...
6,технические обновления биткоине продолжаются ц...
7,резкое снижение стоимости биткоина вызвало сер...
8,биткоин получил критику изза своего воздействи...
9,рыночная стоимость биткоина оставалась относит...


In [16]:
X_new = check_df['headline']

X_new_sequences = tokenizer.texts_to_sequences(X_new)
X_new_padded = pad_sequences(X_new_sequences, maxlen=X_padded.shape[1])

saved_model_path = 'sentiment_model_v5.keras'
model = load_model(saved_model_path)

predictions = model.predict(X_new_padded)

predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

check_df['predicted_sentiment'] = predicted_labels

check_df


1/1 [==============================] - 0s 36ms/step


,headline,predicted_sentiment
0,биткоин упал последние часа вызвав тревогу сре...,negative
1,биткоин достиг нового исторического максимума ...,positive
2,биткоин останется таким году,negative
3,президент франции заявляет концу года купит би...,positive
4,последние новостные сообщения отмечают увеличе...,positive
5,последние колебания цене биткоина вызвали опас...,negative
6,технические обновления биткоине продолжаются ц...,positive
7,резкое снижение стоимости биткоина вызвало сер...,negative
8,биткоин получил критику изза своего воздействи...,negative
9,рыночная стоимость биткоина оставалась относит...,neutral


**Getting current news**

In [17]:
news_time_en = []
news_links_en = []

news_time_ru = []
news_links_ru = []

date_format = "%Y-%m-%d %H:%M:%S%z"

def getNews():
    url_en = "https://api.blockchair.com/news?q=language(en),title(~btc),or,title(~Bitcoin),or,description(~Bitcoin)"
    try:
        response = requests.get(url_en)
        if response.status_code == 200:
            news_data = response.json()
            for i in range(len(news_data['data'])):
                news_time_en.append(str(datetime.strptime(news_data['data'][i]['time'][:-3] + ':00+0000', date_format)))
                news_links_en.append(news_data['data'][i]['link'])
        else:
            print(f"Error: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")
    url_ru = "https://api.blockchair.com/news?q=language(ru),title(~btc),or,title(~Bitcoin),or,description(~Bitcoin)"
    try:
        response = requests.get(url_ru)
        if response.status_code == 200:
            news_data = response.json()
            for i in range(len(news_data['data'])):
                news_time_ru.append(str(datetime.strptime(news_data['data'][i]['time'][:-3] + ':00+0000', date_format)))
                news_links_ru.append(news_data['data'][i]['link'])
        else:
            print(f"Error: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

getNews()

print(news_time_ru)
print(news_links_ru)

print(news_time_en)
print(news_links_en)

['2023-12-20 14:50:00+00:00', '2023-12-20 08:51:00+00:00', '2023-12-19 19:44:00+00:00', '2023-12-19 15:13:00+00:00', '2023-12-19 10:13:00+00:00', '2023-12-19 08:50:00+00:00', '2023-12-18 09:57:00+00:00', '2023-12-18 08:50:00+00:00', '2023-12-16 14:30:00+00:00', '2023-12-15 15:30:00+00:00']
['https://cryptocurrency.tech/majkl-sejlor-zapusk-bitkoin-etf-stanet-glavnym-sobytiem-na-uoll-strit-za-poslednie-30-let/', 'https://forklog.com/news/majkl-sejlor-bitkoin-upadet-do-nulya-ili-vyrastet-do-1-mln', 'https://coinlife.com/news/ssha-vsjo-blizhe-k-odobreniju-bitkoin-etf/', 'https://forklog.com/news/marathon-priobrela-data-tsentry-dlya-majninga-za-178-mln', 'https://cryptocurrency.tech/bitwise-nachali-reklamirovat-svoj-spotovyj-bitkoin-etf/', 'https://forklog.com/news/upravlyayushhij-aktivami-7rcc-podal-zayavku-na-zapusk-bitkoin-etf', 'https://forklog.com/news/ceo-vaneck-nichto-ne-prevzojdet-bitkoin-kak-sredstvo-sberezheniya', 'https://forklog.com/news/bum-ordinals-podbrosil-bitkoin-komissii-d

In [18]:
news_description_en = []
errors_en = []

news_description_ru = []
errors_ru = []

def getNewsDescription():
    for i in range(len(news_links_en)):
        try:
            article = Article(news_links_en[i], language="en")
            article.download()
            article.parse()
            article.nlp()
            news_description_en.append(article.summary)
        except Exception as e:
            print(f"Error processing news link {news_links_en[i]}: {str(e)}")
            errors_en.append(i)
    
    for i in range(len(news_links_ru)):
        try:
            article = Article(news_links_ru[i], language="ru")
            article.download()
            article.parse()
            article.nlp()
            news_description_ru.append(article.summary)
        except Exception as e:
            print(f"Error processing news link {news_links_ru[i]}: {str(e)}")
            errors_ru.append(i)

getNewsDescription()

print(news_description_en)
print(errors_en)

print(news_description_ru)
print(errors_ru)

Error processing news link https://forklog.com/news/marathon-priobrela-data-tsentry-dlya-majninga-za-178-mln: You must `parse()` an article first!
['At this point, popular analysis platform Santiment examined the factors and changing dynamics affecting the course of Bitcoin and the broader crypto market.\nReferring to transaction volumes, social volume and stock market supply at this point, Santiment said that some data point to an upward trend, while some data point to a horizontal and downward trend:“Transaction volumes of Bitcoin and altcoins have slowed down especially in the last week.\nAt the same time, we see that social volume in Bitcoin reached its peak in the first week of December, and social volume in altcoins increased in the following weeks.\nThe decrease in transaction volume and the change in social volume of BTC and altcoins does not necessarily mean that prices have peaked.\n“Overall, there are signals that Bitcoin and altcoins will exhibit sideways movements for a wh

In [19]:
if len(news_time_en) != len(news_description_en):
    for error in errors_en:
        news_time_en.remove(news_time_en[error])

print(len(news_time_en))
print(len(news_description_en))

if len(news_time_ru) != len(news_description_ru):
    for error in errors_ru:
        news_time_ru.remove(news_time_ru[error])

print(len(news_time_ru))
print(len(news_description_ru))

10
10
9
9


In [20]:
try:
    crypto_currencies = yfinance.download('BTC-USD', interval='1m')
except Exception as e:
    print(f"Error processing")

print(crypto_currencies)

[*********************100%%**********************]  1 of 1 completed
                                   Open          High           Low  \
Datetime                                                              
2023-12-13 16:04:00+00:00  41912.917969  41912.917969  41912.917969   
2023-12-13 16:05:00+00:00  41898.082031  41898.082031  41898.082031   
2023-12-13 16:06:00+00:00  41922.726562  41922.726562  41922.726562   
2023-12-13 16:07:00+00:00  41942.429688  41942.429688  41942.429688   
2023-12-13 16:08:00+00:00  41961.804688  41961.804688  41961.804688   
...                                 ...           ...           ...   
2023-12-20 15:57:00+00:00  44053.750000  44053.750000  44053.750000   
2023-12-20 15:58:00+00:00  44034.519531  44034.519531  44034.519531   
2023-12-20 15:59:00+00:00  44018.714844  44018.714844  44018.714844   
2023-12-20 16:00:00+00:00  44024.328125  44024.328125  44024.328125   
2023-12-20 16:01:00+00:00  44053.074219  44053.074219  44053.074219   

       

In [21]:
price_diff_en = []
price_diff_ru = []

for i in range(len(news_description_en)):
    fTargetRow = crypto_currencies[crypto_currencies['Close'].index >= news_time_en[i]].head(1)
    sTargetRow = crypto_currencies[crypto_currencies['Close'].index >= news_time_en[i]].head(11).tail(1)
    fPrice = fTargetRow['Close'].values
    sPrice = sTargetRow['Close'].values

    price_diff_en.append(fPrice[0] - sPrice[0])

print(price_diff_en)

for i in range(len(news_description_ru)):
    fTargetRow = crypto_currencies[crypto_currencies['Close'].index >= news_time_ru[i]].head(1)
    sTargetRow = crypto_currencies[crypto_currencies['Close'].index >= news_time_ru[i]].head(11).tail(1)
    fPrice = fTargetRow['Close'].values
    sPrice = sTargetRow['Close'].values

    price_diff_ru.append(fPrice[0] - sPrice[0])

print(price_diff_ru)

[0.67578125, 41.4296875, 189.8046875, -11.8984375, 19.421875, -32.01953125, -201.33984375, -240.4375, 36.27734375, -44.3046875]
[162.1875, -34.9453125, -26.83203125, 63.4453125, 17.28515625, 129.2578125, 12.5625, -1.15625, 236.640625]


In [22]:
current_news_en = pd.DataFrame({"headline_en": news_description_en, "diff_price_en":price_diff_en})

current_news_en = current_news_en[current_news_en['headline_en'] != ""]

current_news_en = nlp_process(current_news_en)

current_news_en

,headline_en,diff_price_en
0,at this point popular analysis platform santim...,0.675781
1,bitcoin btc broke through the mark for the fir...,41.429688
2,bitcoin price to reach million dollarsin addit...,189.804688
4,bitcoins market cap increased by a peak of in ...,19.421875
5,followed by viewers the famous crypto analyst ...,-32.019531
6,fox business network senior reporter charles g...,-201.339844
7,the dogecoin price has been put in a perilous ...,-240.437500
8,with the decline it experienced yesterday even...,36.277344
9,blackrock spot bitcoin etf gains momentum with...,-44.304688


In [23]:
current_news_ru = pd.DataFrame({"headline_ru": news_description_ru, "diff_price_ru":price_diff_ru})

current_news_ru = current_news_ru[current_news_ru['headline_ru'] != ""]

current_news_ru = nlp_process_ru(current_news_ru)

current_news_ru

,headline_ru,diff_price_ru
0,бывший глава компании microstrategy майкл сэйл...,162.187500
1,майкл сэйлор биткоин упадет нуля вырастет млнр...,-34.945312
2,sec меняют риторику фоне проигрыша судебных де...,-26.832031
3,сети появился видеоролик рекламирующий спотовы...,63.445312
4,управляющий активами rcc подал заявку запуск б...,17.285156
5,ceo vaneck ничто превзойдет биткоин средство с...,129.257812
6,бум ordinals подбросил биткоинкомиссии рекордн...,12.562500
7,верните тона вейса коррекция cезон альтов укра...,-1.156250
8,биткоин растет доллар падает итоги года прогно...,236.640625


In [24]:
current_news_ru

,headline_ru,diff_price_ru
0,бывший глава компании microstrategy майкл сэйл...,162.187500
1,майкл сэйлор биткоин упадет нуля вырастет млнр...,-34.945312
2,sec меняют риторику фоне проигрыша судебных де...,-26.832031
3,сети появился видеоролик рекламирующий спотовы...,63.445312
4,управляющий активами rcc подал заявку запуск б...,17.285156
5,ceo vaneck ничто превзойдет биткоин средство с...,129.257812
6,бум ordinals подбросил биткоинкомиссии рекордн...,12.562500
7,верните тона вейса коррекция cезон альтов укра...,-1.156250
8,биткоин растет доллар падает итоги года прогно...,236.640625


In [25]:
current_news_en

,headline_en,diff_price_en
0,at this point popular analysis platform santim...,0.675781
1,bitcoin btc broke through the mark for the fir...,41.429688
2,bitcoin price to reach million dollarsin addit...,189.804688
4,bitcoins market cap increased by a peak of in ...,19.421875
5,followed by viewers the famous crypto analyst ...,-32.019531
6,fox business network senior reporter charles g...,-201.339844
7,the dogecoin price has been put in a perilous ...,-240.437500
8,with the decline it experienced yesterday even...,36.277344
9,blackrock spot bitcoin etf gains momentum with...,-44.304688


In [26]:
ru_current_df = current_news_ru[['headline_ru', 'diff_price_ru']]
ru_current_df

,headline_ru,diff_price_ru
0,бывший глава компании microstrategy майкл сэйл...,162.187500
1,майкл сэйлор биткоин упадет нуля вырастет млнр...,-34.945312
2,sec меняют риторику фоне проигрыша судебных де...,-26.832031
3,сети появился видеоролик рекламирующий спотовы...,63.445312
4,управляющий активами rcc подал заявку запуск б...,17.285156
5,ceo vaneck ничто превзойдет биткоин средство с...,129.257812
6,бум ordinals подбросил биткоинкомиссии рекордн...,12.562500
7,верните тона вейса коррекция cезон альтов укра...,-1.156250
8,биткоин растет доллар падает итоги года прогно...,236.640625


In [27]:
en_current_df = current_news_en[['headline_en', 'diff_price_en']]
en_current_df

,headline_en,diff_price_en
0,at this point popular analysis platform santim...,0.675781
1,bitcoin btc broke through the mark for the fir...,41.429688
2,bitcoin price to reach million dollarsin addit...,189.804688
4,bitcoins market cap increased by a peak of in ...,19.421875
5,followed by viewers the famous crypto analyst ...,-32.019531
6,fox business network senior reporter charles g...,-201.339844
7,the dogecoin price has been put in a perilous ...,-240.437500
8,with the decline it experienced yesterday even...,36.277344
9,blackrock spot bitcoin etf gains momentum with...,-44.304688


In [28]:
current_news = pd.DataFrame({"headline": pd.concat([ru_current_df['headline_ru'], en_current_df['headline_en']], ignore_index=True),
                             "diff_price": pd.concat([ru_current_df['diff_price_ru'], en_current_df['diff_price_en']], ignore_index=True)})
current_news

,headline,diff_price
0,бывший глава компании microstrategy майкл сэйл...,162.187500
1,майкл сэйлор биткоин упадет нуля вырастет млнр...,-34.945312
2,sec меняют риторику фоне проигрыша судебных де...,-26.832031
3,сети появился видеоролик рекламирующий спотовы...,63.445312
4,управляющий активами rcc подал заявку запуск б...,17.285156
5,ceo vaneck ничто превзойдет биткоин средство с...,129.257812
6,бум ordinals подбросил биткоинкомиссии рекордн...,12.562500
7,верните тона вейса коррекция cезон альтов укра...,-1.156250
8,биткоин растет доллар падает итоги года прогно...,236.640625
9,at this point popular analysis platform santim...,0.675781


In [29]:
X_new = current_news['headline']

X_new_sequences = tokenizer.texts_to_sequences(X_new)
X_new_padded = pad_sequences(X_new_sequences, maxlen=X_padded.shape[1])

saved_model_path = 'sentiment_model_v5.keras'
model = load_model(saved_model_path)

predictions = model.predict(X_new_padded)

predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

current_news['predicted_sentiment'] = predicted_labels

current_news

1/1 [==============================] - 0s 33ms/step


,headline,diff_price,predicted_sentiment
0,бывший глава компании microstrategy майкл сэйл...,162.187500,neutral
1,майкл сэйлор биткоин упадет нуля вырастет млнр...,-34.945312,positive
2,sec меняют риторику фоне проигрыша судебных де...,-26.832031,positive
3,сети появился видеоролик рекламирующий спотовы...,63.445312,neutral
4,управляющий активами rcc подал заявку запуск б...,17.285156,positive
5,ceo vaneck ничто превзойдет биткоин средство с...,129.257812,positive
6,бум ordinals подбросил биткоинкомиссии рекордн...,12.562500,positive
7,верните тона вейса коррекция cезон альтов укра...,-1.156250,positive
8,биткоин растет доллар падает итоги года прогно...,236.640625,positive
9,at this point popular analysis platform santim...,0.675781,positive


In [30]:
def analyze_news_effect(predicted_sentiment, diff_price):
    if diff_price > 0 and predicted_sentiment == 'positive':
        return "The news affected the price: positive effect"
    elif diff_price < 0 and predicted_sentiment == 'negative':
        return "The news affected the price: negative effect"
    elif -5 <= diff_price <= 5 and predicted_sentiment == 'neutral':
        return "The news did not affect the price: neutral effect"
    elif diff_price < 0 and predicted_sentiment == 'positive':
        return "The news did not affect the price: negative effect"
    elif diff_price > 0 and predicted_sentiment == 'negative':
        return "The news did not affect the price: positive effect"
    elif predicted_sentiment == 'neutral' and diff_price > 0:
        return "The news did not affect the price: positive effect"
    elif predicted_sentiment == 'neutral' and diff_price < 0:
        return "The news did not affect the price: negative effect"

In [31]:
current_news['relationship'] = current_news.apply(lambda row: analyze_news_effect(row['predicted_sentiment'], row['diff_price']), axis=1)
current_news

,headline,diff_price,predicted_sentiment,relationship
0,бывший глава компании microstrategy майкл сэйл...,162.187500,neutral,The news did not affect the price: positive ef...
1,майкл сэйлор биткоин упадет нуля вырастет млнр...,-34.945312,positive,The news did not affect the price: negative ef...
2,sec меняют риторику фоне проигрыша судебных де...,-26.832031,positive,The news did not affect the price: negative ef...
3,сети появился видеоролик рекламирующий спотовы...,63.445312,neutral,The news did not affect the price: positive ef...
4,управляющий активами rcc подал заявку запуск б...,17.285156,positive,The news affected the price: positive effect
5,ceo vaneck ничто превзойдет биткоин средство с...,129.257812,positive,The news affected the price: positive effect
6,бум ordinals подбросил биткоинкомиссии рекордн...,12.562500,positive,The news affected the price: positive effect
7,верните тона вейса коррекция cезон альтов укра...,-1.156250,positive,The news did not affect the price: negative ef...
8,биткоин растет доллар падает итоги года прогно...,236.640625,positive,The news affected the price: positive effect
9,at this point popular analysis platform santim...,0.675781,positive,The news affected the price: positive effect
